In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df= pd.read_excel('1- DoE and Data.xlsx')

In [3]:
df.head()

,StdOrder,RunOrder,Layer Height,Exposure Time,Light Off Delay,Orientation,Time,Diameter,Length,Height 1,Square Pin Width,Square Pin Length,Height 2,Surface Roughness
0,35,1,0.050,2.25,12,Z,137,4.077,12.035,3.080,2.005,2.125,7.040,2.2
1,10,2,0.075,2.50,12,X,93,4.202,11.935,3.005,2.015,1.975,7.055,1.9
2,2,3,0.100,2.00,13,X,74,4.128,12.020,3.110,2.115,2.070,7.090,2.5
3,31,4,0.050,2.00,13,Z,143,4.148,11.970,3.100,1.975,2.070,7.010,2.0
4,36,5,0.100,2.25,12,Z,70,3.885,12.150,3.215,1.860,2.045,7.010,5.2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   StdOrder           45 non-null     int64  
 1   RunOrder           45 non-null     int64  
 2   Layer Height       45 non-null     float64
 3   Exposure Time      45 non-null     float64
 4   Light Off Delay    45 non-null     int64  
 5   Orientation        45 non-null     object 
 6   Time               45 non-null     int64  
 7   Diameter           45 non-null     float64
 8   Length             45 non-null     float64
 9   Height 1           45 non-null     float64
 10  Square Pin Width   45 non-null     float64
 11  Square Pin Length  45 non-null     float64
 12  Height 2           45 non-null     float64
 13  Surface Roughness  45 non-null     float64
dtypes: float64(9), int64(4), object(1)
memory usage: 5.0+ KB


In [5]:
np.sum(df.isna())

StdOrder             0
RunOrder             0
Layer Height         0
Exposure Time        0
Light Off Delay      0
Orientation          0
Time                 0
Diameter             0
Length               0
Height 1             0
Square Pin Width     0
Square Pin Length    0
Height 2             0
Surface Roughness    0
dtype: int64

In [6]:
df.columns

Index(['StdOrder', 'RunOrder', 'Layer Height', 'Exposure Time',
       'Light Off Delay', 'Orientation', 'Time', 'Diameter', 'Length',
       'Height 1', 'Square Pin Width', 'Square Pin Length', 'Height 2',
       'Surface Roughness'],
      dtype='object')

In [7]:
df.drop(['StdOrder', 'RunOrder','Time'],axis=1,inplace=True)

In [8]:
df1=df.copy()
df1['Diameter']=df1['Diameter']-4
df1['Length']=df1['Length']-12
df1['Height 1']=df1['Height 1']-3
df1['Square Pin Width']=df1['Square Pin Width']-2
df1['Square Pin Length']=df1['Square Pin Length']-2
df1['Height 2']=df1['Height 2']-7


In [9]:
df1.head()

,Layer Height,Exposure Time,Light Off Delay,Orientation,Diameter,Length,Height 1,Square Pin Width,Square Pin Length,Height 2,Surface Roughness
0,0.050,2.25,12,Z,0.077,0.035,0.080,0.005,0.125,0.040,2.2
1,0.075,2.50,12,X,0.202,-0.065,0.005,0.015,-0.025,0.055,1.9
2,0.100,2.00,13,X,0.128,0.020,0.110,0.115,0.070,0.090,2.5
3,0.050,2.00,13,Z,0.148,-0.030,0.100,-0.025,0.070,0.010,2.0
4,0.100,2.25,12,Z,-0.115,0.150,0.215,-0.140,0.045,0.010,5.2


In [10]:
cols= ['Diameter', 'Length','Height 1', 'Square Pin Width', 'Square Pin Length', 'Height 2']
for i in cols:
    try:
        df1.drop(df1[df1[i]==0].index[0],axis=0,inplace=True)
    except:
        df1

In [11]:
df1.rename(columns={'Layer Height':'lr_h','Exposure Time':'exp_t','Light Off Delay':'lod','Orientation':'orient',
                   'Diameter':'D1','Length':'L1','Height 1':'H1', 'Square Pin Width':'S1', 'Square Pin Length':'S2'
                    , 'Height 2':'H2','Surface Roughness':'surf_R'},inplace=True)

In [12]:
df1.columns

Index(['lr_h', 'exp_t', 'lod', 'orient', 'D1', 'L1', 'H1', 'S1', 'S2', 'H2',
       'surf_R'],
      dtype='object')

# Linear Regression model 
### Input parameters are - variation in Diameter, Length, Height, and Square Pin Height and Width, Surface Roughness and Orientation
### Output variable is Exposure Time

In [15]:
from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.exp_t

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [16]:
surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr_model =LinearRegression()
for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        lr_model.fit(train_data_1,y_train)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,lr_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

RMSE Value:  0.05805695642589079 Removed by this columns:  D1
RMSE Value:  0.09573443238309452 Removed by this columns:  L1
RMSE Value:  0.10495404796991609 Removed by this columns:  H1
RMSE Value:  0.10174224888403405 Removed by this columns:  S1
RMSE Value:  0.10350598195649863 Removed by this columns:  S2
RMSE Value:  0.10959180659904738 Removed by this columns:  H2


RMSE Value:  0.045180689984254174 Removed by this columns:  L1
RMSE Value:  0.055951067267391345 Removed by this columns:  H1
RMSE Value:  0.05811544402799883 Removed by this columns:  S1
RMSE Value:  0.05766945152929629 Removed by this columns:  S2
RMSE Value:  0.06162108206169773 Removed by this columns:  H2


RMSE Value:  0.04570854484894744 Removed by this columns:  H1
RMSE Value:  0.044757964660957834 Removed by this columns:  S1
RMSE Value:  0.044957862337540636 Removed by this columns:  S2
RMSE Value:  0.051980452801355534 Removed by this columns:  H2


RMSE Value:  0.04074101073804467 Removed by this columns:  

#### From this result for linear Regression model best variation parameter is **Height 2**, Who have major influences on **Exposure Time** with respect to low value of RMSE.

# Ridge Regression model 
### Input parameters are - variation in Diameter, Length, Height, and Square Pin Height and Width, Surface Roughness and Orientation
### Output variable is Exposure Time

In [17]:
from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.exp_t

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
ridge_model =Ridge()
for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        ridge_model.fit(train_data_1,y_train)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,ridge_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

RMSE Value:  0.04194940073419565 Removed by this columns:  D1
RMSE Value:  0.04212779331348823 Removed by this columns:  L1
RMSE Value:  0.04266805886727522 Removed by this columns:  H1
RMSE Value:  0.042159569499217876 Removed by this columns:  S1
RMSE Value:  0.042601741022385314 Removed by this columns:  S2
RMSE Value:  0.04358944540352265 Removed by this columns:  H2


RMSE Value:  0.041414962109861855 Removed by this columns:  L1
RMSE Value:  0.041911643564324294 Removed by this columns:  H1
RMSE Value:  0.041358414363955556 Removed by this columns:  S1
RMSE Value:  0.0418437489943411 Removed by this columns:  S2
RMSE Value:  0.04275251591624388 Removed by this columns:  H2


RMSE Value:  0.04079518343050789 Removed by this columns:  L1
RMSE Value:  0.041292875862829065 Removed by this columns:  H1
RMSE Value:  0.04122031364851843 Removed by this columns:  S2
RMSE Value:  0.04208280534289649 Removed by this columns:  H2


RMSE Value:  0.04073850095146282 Removed by this columns:  

# KNN Model

In [18]:
from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.exp_t


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
KNN_model =KNeighborsRegressor()
for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        KNN_model.fit(train_data_1,y_train)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,KNN_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

RMSE Value:  0.05277777777777777 Removed by this columns:  D1
RMSE Value:  0.05277777777777777 Removed by this columns:  L1
RMSE Value:  0.05277777777777777 Removed by this columns:  H1
RMSE Value:  0.05277777777777777 Removed by this columns:  S1
RMSE Value:  0.05277777777777777 Removed by this columns:  S2
RMSE Value:  0.049166666666666636 Removed by this columns:  H2


RMSE Value:  0.049166666666666636 Removed by this columns:  D1
RMSE Value:  0.05277777777777777 Removed by this columns:  L1
RMSE Value:  0.049166666666666636 Removed by this columns:  H1
RMSE Value:  0.049166666666666636 Removed by this columns:  S1
RMSE Value:  0.049166666666666636 Removed by this columns:  S2


RMSE Value:  0.046111111111111096 Removed by this columns:  L1
RMSE Value:  0.049166666666666636 Removed by this columns:  H1
RMSE Value:  0.049166666666666636 Removed by this columns:  S1
RMSE Value:  0.049166666666666636 Removed by this columns:  S2


RMSE Value:  0.046111111111111096 Removed by this colum

# DNN Model

In [25]:
import tensorflow as tf
from tensorflow import keras


from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.exp_t

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')


for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        dnn_model= keras.Sequential([keras.layers.Dense((len(x_train.columns)+1),input_shape=((len(x_train.columns)+1),),activation='relu'),
                                     keras.layers.Dense(1,activation='sigmoid')])

        dnn_model.compile(optimizer='adam',
                          loss='binary_crossentropy')
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        dnn_model.fit(train_data_1,y_train,epochs=500,verbose=False)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,dnn_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

1/1 [==============================] - 0s 39ms/step
RMSE Value:  1.7430581053178023 Removed by this columns:  D1
1/1 [==============================] - 0s 38ms/step
RMSE Value:  1.7430635823396672 Removed by this columns:  L1
1/1 [==============================] - 0s 39ms/step
RMSE Value:  1.7430559164948904 Removed by this columns:  H1
1/1 [==============================] - 0s 39ms/step
RMSE Value:  1.7430631817059323 Removed by this columns:  S1
1/1 [==============================] - 0s 38ms/step
RMSE Value:  1.7430555721124017 Removed by this columns:  S2
1/1 [==============================] - 0s 39ms/step
RMSE Value:  1.7430624597850115 Removed by this columns:  H2


1/1 [==============================] - 0s 38ms/step
RMSE Value:  1.7430665559772136 Removed by this columns:  D1
1/1 [==============================] - 0s 41ms/step
RMSE Value:  1.7443053604975032 Removed by this columns:  L1
1/1 [==============================] - 0s 39ms/step
RMSE Value:  1.743058870241172 Removed by 

# Linear Regression model 
### Input parameters are - variation in Diameter, Length, Height, and Square Pin Height and Width, Surface Roughness and Orientation
### Output variable is Light off Delay

In [28]:
from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.lod

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr_model =LinearRegression()
for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        lr_model.fit(train_data_1,y_train)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,lr_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

RMSE Value:  0.6850876553840695 Removed by this columns:  D1
RMSE Value:  0.33673960962122645 Removed by this columns:  L1
RMSE Value:  0.36079729941833794 Removed by this columns:  H1
RMSE Value:  0.3381248007422209 Removed by this columns:  S1
RMSE Value:  0.3915467483962486 Removed by this columns:  S2
RMSE Value:  0.28452409714179544 Removed by this columns:  H2


RMSE Value:  0.5330532388036053 Removed by this columns:  D1
RMSE Value:  0.3047608636151347 Removed by this columns:  L1
RMSE Value:  0.27785916868019916 Removed by this columns:  H1
RMSE Value:  0.2849356949163206 Removed by this columns:  S1
RMSE Value:  0.307816353628379 Removed by this columns:  S2


RMSE Value:  0.5675529950031831 Removed by this columns:  D1
RMSE Value:  0.30581196444242426 Removed by this columns:  L1
RMSE Value:  0.27864538673303707 Removed by this columns:  S1
RMSE Value:  0.3068012847901618 Removed by this columns:  S2


RMSE Value:  0.540457615900831 Removed by this columns:  D1
RMSE Value:  0

# Ridge regression

In [30]:
from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.lod

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
ridge_model =Ridge()
for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        ridge_model.fit(train_data_1,y_train)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,ridge_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

RMSE Value:  0.4799889288068982 Removed by this columns:  D1
RMSE Value:  0.46852297520165664 Removed by this columns:  L1
RMSE Value:  0.4720398909367296 Removed by this columns:  H1
RMSE Value:  0.47124117565150136 Removed by this columns:  S1
RMSE Value:  0.4708580972162966 Removed by this columns:  S2
RMSE Value:  0.467228368292247 Removed by this columns:  H2


RMSE Value:  0.4765573520725926 Removed by this columns:  D1
RMSE Value:  0.4651659104685464 Removed by this columns:  L1
RMSE Value:  0.4684785462245118 Removed by this columns:  H1
RMSE Value:  0.46762430678324207 Removed by this columns:  S1
RMSE Value:  0.4672128681154893 Removed by this columns:  S2


RMSE Value:  0.4742345006363531 Removed by this columns:  D1
RMSE Value:  0.4663901755611909 Removed by this columns:  H1
RMSE Value:  0.4655853688173423 Removed by this columns:  S1
RMSE Value:  0.46515470606384857 Removed by this columns:  S2


RMSE Value:  0.47421426771176795 Removed by this columns:  D1
RMSE Value:  0

# KNN Model

In [32]:
from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.lod


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
KNN_model =KNeighborsRegressor()
for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        KNN_model.fit(train_data_1,y_train)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,KNN_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

RMSE Value:  0.5999999999999999 Removed by this columns:  D1
RMSE Value:  0.5511111111111111 Removed by this columns:  L1
RMSE Value:  0.5511111111111111 Removed by this columns:  H1
RMSE Value:  0.5511111111111111 Removed by this columns:  S1
RMSE Value:  0.5999999999999999 Removed by this columns:  S2
RMSE Value:  0.6133333333333332 Removed by this columns:  H2


RMSE Value:  0.5511111111111111 Removed by this columns:  D1
RMSE Value:  0.5511111111111111 Removed by this columns:  H1
RMSE Value:  0.5511111111111111 Removed by this columns:  S1
RMSE Value:  0.5511111111111111 Removed by this columns:  S2
RMSE Value:  0.5511111111111111 Removed by this columns:  H2


RMSE Value:  0.5511111111111111 Removed by this columns:  H1
RMSE Value:  0.5511111111111111 Removed by this columns:  S1
RMSE Value:  0.5999999999999999 Removed by this columns:  S2
RMSE Value:  0.5644444444444444 Removed by this columns:  H2


RMSE Value:  0.5511111111111111 Removed by this columns:  S1
RMSE Value:  0.551

# DNN Model

In [31]:
import tensorflow as tf
from tensorflow import keras


from sklearn.preprocessing import LabelEncoder
df1['orient']=LabelEncoder().fit_transform(df1['orient'])
x= df1[['D1','L1', 'H1', 'S1', 'S2', 'H2','orient','surf_R']]
y= df1.lod

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

surf_R_train_df=x_train[['surf_R','orient']]
x_train.drop(['surf_R','orient'],axis=1,inplace=True)
surf_R_test_df=x_test[['surf_R','orient']]
x_test.drop(['surf_R','orient'],axis=1,inplace=True)

import warnings
warnings.filterwarnings('ignore')


for j in range(len(x_train.columns)-1):
    mse=[]
    for i in range(len(x_train.columns)):
        dnn_model= keras.Sequential([keras.layers.Dense((len(x_train.columns)+1),input_shape=((len(x_train.columns)+1),),activation='relu'),
                                     keras.layers.Dense(1,activation='sigmoid')])

        dnn_model.compile(optimizer='adam',
                          loss='binary_crossentropy')
        train_data=x_train.drop([x_train.columns[i]],axis=1)
        train_data_1= pd.concat([train_data,surf_R_train_df],axis=1)
        dnn_model.fit(train_data_1,y_train,epochs=500,verbose=False)
        
        test_data= x_test.drop([x_test.columns[i]],axis=1)
        test_data_1= pd.concat([test_data,surf_R_test_df],axis=1)
        MSE=mean_squared_error(y_test,dnn_model.predict(test_data_1))
        print('RMSE Value: ', MSE,'Removed by this columns: ',x_train.columns[i])
        mse.append(MSE)
    print('\n')
    mse.index(np.min(mse))
    x_train.drop([x_train.columns[mse.index(np.min(mse))]],axis=1,inplace=True)
    x_test.drop([x_test.columns[mse.index(np.min(mse))]],axis=1,inplace=True)

1/1 [==============================] - 0s 39ms/step
RMSE Value:  141.66667962074496 Removed by this columns:  D1
1/1 [==============================] - 0s 41ms/step
RMSE Value:  141.66931884760112 Removed by this columns:  L1
1/1 [==============================] - 0s 38ms/step
RMSE Value:  150.79758613735123 Removed by this columns:  H1
1/1 [==============================] - 0s 41ms/step
RMSE Value:  141.86142946377225 Removed by this columns:  S1
1/1 [==============================] - 0s 40ms/step
RMSE Value:  141.66747905570088 Removed by this columns:  S2
1/1 [==============================] - 0s 39ms/step
RMSE Value:  145.40904740448707 Removed by this columns:  H2


1/1 [==============================] - 0s 41ms/step
RMSE Value:  141.66987953011753 Removed by this columns:  L1
1/1 [==============================] - 0s 47ms/step
RMSE Value:  142.20287634709172 Removed by this columns:  H1
1/1 [==============================] - 0s 41ms/step
RMSE Value:  141.68495305689666 Removed by